<a href="https://colab.research.google.com/github/nguforche/LLaMPS/blob/main/Gradio_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r "/content/drive/My Drive/your-model/" "/content/your-model/"

In [ ]:
!pip install gradio

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/your-model/",
    max_seq_length = 8192,
    dtype = None,
    load_in_4bit = False,
)

In [ ]:
import gradio as gr
import transformers
import torch
from threading import Thread
from gradio.themes.utils.colors import Color

text_color = "#FFFFFF"
app_background = "#0A0A0A"
user_inputs_background = "#193C4C"
widget_bg = "#000100"
button_bg = "#141414"

DESCRIPTION = """
# chat
"""

def chat(user_input, history):
    messages = []
    for pair in history:
        messages.append({'role': 'user', 'content': pair[0]})
        messages.append({'role': 'assistant', 'content': pair[1]})
    messages.append({'role': 'user', 'content': user_input})

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    device = 'cuda'
    model_inputs = {'input_ids': encodeds.to(device)}

    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)

    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=8192,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
        #top_p=0.8,
        #temperature=0.85,
        #top_k=50
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    model_output = ""
    for new_text in streamer:
        model_output += new_text
        yield model_output
    return model_output


with gr.Blocks(theme=gr.themes.Monochrome(font=[gr.themes.GoogleFont("Roboto"), "Arial", "sans-serif"])) as demo:  #main.
    gr.Markdown(DESCRIPTION)
    chatbot = gr.ChatInterface(fn=chat)

demo.queue()
demo.launch(debug=True)